In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
import librosa as librosa
import warnings
warnings.filterwarnings('ignore')
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
%matplotlib inline
import scipy.stats as scs
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
import itertools
from sklearn.preprocessing import StandardScaler # data normalization
from sklearn.metrics import precision_score # evaluation metric
from sklearn.metrics import accuracy_score # evaluation metric
from sklearn.metrics import recall_score # evaluation metric
from sklearn.metrics import classification_report # evaluation metric
from sklearn.metrics import confusion_matrix # evaluation metric
from sklearn.metrics import log_loss # evaluation metric
from matplotlib import rcParams # plot size customization
rcParams['figure.figsize'] = (20, 10)

In [11]:
path = '/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/src'
pat = '/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data'

In [14]:
os.chdir(pat)
df2 = pd.read_pickle('original_df.pickle')
df = pd.read_pickle('MusicFeats.pickle')
os.chdir(path)
df.head()


,genre,mfcc,chroma_sftf,flux,rolloff,zero_crossing_rate,contrast,flatness,rmse,tempo
0,hip-hop,4.450824,0.431049,1.365420,4318.922711,0.068300,19.013011,0.025153,0.345805,151.999081
1,hip-hop,-0.377781,0.511142,1.667078,4419.694105,0.062677,18.660331,0.022863,0.176869,123.046875
2,hip-hop,0.259693,0.522985,1.359143,5745.967884,0.159102,20.103879,0.053650,0.276045,95.703125
3,hip-hop,0.297143,0.399623,1.744030,4064.054853,0.060242,21.550626,0.020770,0.365921,95.703125
4,hip-hop,-3.304997,0.288411,1.144687,1368.274351,0.039064,21.798417,0.000360,0.072659,117.453835


In [15]:
y = df["genre"]
X = df.drop('genre',axis=1,inplace=False)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
lr = LogisticRegression(C = 1000, solver = 'liblinear')
lr.fit(X_train,y_train)
yhat = lr.predict(X_test)
yhat_prob = lr.predict_proba(X_test)

print("Accuracy Score : ", accuracy_score(yhat,y_test))

print("Precision Score : ",precision_score(y_test, yhat, 
                                           pos_label='positive',
                                           average='macro'))
print("Recall Score : ",recall_score(y_test, yhat, 
                                           pos_label='positive',
                                           average='macro'))

Accuracy Score :  0.5262605042016807
Precision Score :  0.5007435920849458
Recall Score :  0.5336156929939722


In [12]:
'''
gen = df.loc[:,"genre"]
dums = pd.get_dummies(gen,drop_first=True)
df.drop('genre',axis=1,inplace=True)
dums.head()
df = pd.concat([df,dums],axis=1)

df.head()
'''

'\ngen = df.loc[:,"genre"]\ndums = pd.get_dummies(gen,drop_first=True)\ndf.drop(\'genre\',axis=1,inplace=True)\ndums.head()\ndf = pd.concat([df,dums],axis=1)\n\ndf.head()\n'

In [67]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
model.fit(X_train,y_train)

#Predict Output
y_pred= model.predict(X_test)


In [68]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.37969924812030076


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# scores = []
# for i in range(100):
#     clf = RandomForestClassifier(n_estimators=15)

#     # Train the model using the training sets
#     clf.fit(X_train,y_train)

#     #Predict Output
#     y_pred= clf.predict(X_test)
#     scores.append(metrics.accuracy_score(y_test, y_pred))

# print(np.mean(scores))

from sklearn.model_selection import GridSearchCV# Create the parameter grid based on the results of random search 

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}# Create a based model
rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
# Fit the grid search to the data

grid_search.fit(train_features, train_labels)

grid_search.best_params_{'bootstrap': True,
 'max_depth': 80,
 'max_features': 3,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 100}

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)


print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))
                  

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [60]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)  # almost always use shuffle=True
fold_scores = []

for train, test in kf.split(X):
    model = LogisticRegression()
    model.fit(X.values[train], y.values[train])
    fold_scores.append(model.score(X.values[test], y.values[test]))
    
print(np.mean(fold_scores))

0.4698999025600141


baseline is the probability of randomly sleecting a song...

baseline is about 20%

In [112]:
tests = ['KNeighborsClassifier','LogisticRegression','RidgeClassifier','BaggingClassifier']

scores = ['Model','Best Score','Mean','Std. Dev','Params']
scores_per_test = pd.DataFrame(columns=[scores])
scores_per_test['Model'] = tests
 
scores_per_test.head()

mean_scores = []
std_devs = []

In [113]:
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


scores_per_test.iloc[1,1] = grid_result.best_score_
scores_per_test.iloc[1,2] = sum(means) / len(means)
scores_per_test.iloc[1,3] = sum(stds) / len(stds)
scores_per_test.iloc[1,4] = params


Best: 0.618577 using {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.618577 (0.019213) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.438987 (0.021217) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.534651 (0.023326) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.617350 (0.020320) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.437848 (0.020209) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.532806 (0.022669) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.600003 (0.022655) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.438636 (0.021347) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.522212 (0.024838) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.568993 (0.022594) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.439775 (0.020023) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.503463 (0.024929) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.534215 (0.021816) wit

In [114]:
mean_scores.append(means)
std_devs.append(stds)

In [115]:
from sklearn.linear_model import RidgeClassifier

model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# define grid search
grid = dict(alpha=alpha)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
    

scores_per_test.iloc[2,1] = grid_result.best_score_
scores_per_test.iloc[2,2] = sum(means) / len(means)
scores_per_test.iloc[2,3] = sum(stds) / len(stds)
scores_per_test.iloc[2,4] = params

Best: 0.593522 using {'alpha': 0.1}
0.593522 (0.023155) with: {'alpha': 0.1}
0.590280 (0.023471) with: {'alpha': 0.2}
0.588179 (0.022596) with: {'alpha': 0.3}
0.586602 (0.023089) with: {'alpha': 0.4}
0.586164 (0.023288) with: {'alpha': 0.5}
0.585902 (0.022843) with: {'alpha': 0.6}
0.584938 (0.022772) with: {'alpha': 0.7}
0.584412 (0.022712) with: {'alpha': 0.8}
0.583362 (0.022456) with: {'alpha': 0.9}
0.583099 (0.023081) with: {'alpha': 1.0}


In [116]:
mean_scores.append(means)
std_devs.append(stds)

In [117]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier()

n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
    
scores_per_test.iloc[0,1] = grid_result.best_score_
scores_per_test.iloc[0,2] = sum(means) / len(means)
scores_per_test.iloc[0,3] = sum(stds) / len(stds)
scores_per_test.iloc[0,4] = params

Best: 0.438373 using {'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
0.361360 (0.021811) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
0.361360 (0.021811) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'distance'}
0.377839 (0.021287) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
0.378011 (0.019385) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
0.392733 (0.018872) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
0.389930 (0.022377) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
0.405084 (0.024246) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
0.399125 (0.022306) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
0.408590 (0.024897) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
0.407888 (0.023121) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
0.410167

In [118]:
mean_scores.append(means)
std_devs.append(stds)

In [119]:
from sklearn.ensemble import BaggingClassifier

# define models and parameters
model = BaggingClassifier()
n_estimators = [10, 100, 1000]
# define grid search
grid = dict(n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    


scores_per_test.iloc[3,1] = grid_result.best_score_
scores_per_test.iloc[3,2] = sum(means) / len(means)
scores_per_test.iloc[3,3] = sum(stds) / len(stds)
scores_per_test.iloc[3,4] = params

Best: 0.648795 using {'n_estimators': 1000}
0.617426 (0.018526) with: {'n_estimators': 10}
0.643365 (0.020245) with: {'n_estimators': 100}
0.648795 (0.022919) with: {'n_estimators': 1000}


In [124]:
mean_scores.append(means)
std_devs.append(stds)

dd = pd.Series(mean_scores)



In [89]:
scores_per_test.head()

,Model,Best Score,Mean,Std. Dev,Params
0,KNeighborsClassifier,0.438373,0.405715,0.02277,"[{'metric': 'euclidean', 'n_neighbors': 1, 'we..."
1,LogisticRegression,0.618577,0.513264,0.021847,"[{'C': 100, 'penalty': 'l2', 'solver': 'newton..."
2,RidgeClassifier,0.593522,0.586646,0.022946,"[{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3..."
3,BaggingClassifier,0.648358,0.636006,0.022437,"[{'n_estimators': 10}, {'n_estimators': 100}, ..."


In [91]:
print(os.getcwd())
pat = '/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data'
os.chdir(pat)
scores_per_test.to_pickle('scores1.pickle')

scores_per_test.to_csv('scores1')

/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data
